In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import math
from PIL import Image

import sylvester
import Robot_Matrix
# termination criteria
# cv2.TERM_CRITERIA_EPS : 정해둔 오차범위내에 다다르면, 알고리즘을 멈춘다.
# cv2.TERM_CRITERIA_MAX_ITER : 지정한 횟수에 다다르면, 알고리즘을 멈춘다.
#(알고리즘 제동조건, 횟수, 오차범위)

w = int(input("체스판 가로의 꼭지점"))
h = int(input("체스판 세로의 꼭지점"))

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# object point들을 미리 설정

#objp = np.zeros((체스판 가로, 체스판 세로)) 배열 선언 및 초기화
objp = np.zeros((w*h,3), np.float32)

#objp의 모든행의 [0:2]인덱스 부분에 번호를 부여
objp[:,:2] = np.mgrid[0:h,0:w].T.reshape(-1,2)

# 모든 이미지에서 객체 포인트와 이미지 포인트를 저장하는 배열
objpoints = [] # 실제 세상에서의 3d 포인트들을 넣을 배열
imgpoints = [] # image plane에서의 2d 포인트들을 넣을 배열

#png 파일들이 있으면, 다시, JPEG 파일로 재생성
imp = glob.glob('./calibration_data/*.png')

i = 0

for fname in imp:
    i = i + 1
    im = Image.open(fname)
    im.save("{0}.jpg".format(i), "JPEG")
    
images = glob.glob('./calibration_data/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    
    #흑백으로 변경
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 체스보드 코너들을 찾는 부분
    ret, corners = cv2.findChessboardCorners(gray, (6,4),None)
    
    # 만약, 찾았으면, objpoints, imgpoints에 추가
    if ret == True:
        objpoints.append(objp)
        
        #입력 영상 image에서 검출된 코너점 corners를 입력하여 코너점의 위치를 부화소 수준으로 다시 계산하여 반환한다.
        #cv2.cornerSubPix(image, corners, winSize, zeroZone, criteria)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        
        # 검출한 코너들을 보여주는 부분, 현재, 응답오류로 인해서 제외시켰다.
        img = cv2.drawChessboardCorners(img, (h,w), corners2,ret)
        
        img = cv2.resize(img, dsize=(640, 480), interpolation=cv2.INTER_AREA)

        #cv2.imshow('img', img)
        #cv2.waitKey(1000)

cv2.destroyAllWindows()

체스판 가로의 꼭지점4
체스판 세로의 꼭지점6


In [2]:
# 내부 파라미터 : camera calibration 결과, cameramatrix, 왜곡계수, 회전벡터, 병진벡터
_, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [3]:
# 외부 파라미터 : pose estimation 결과, 회전벡터, 병진벡터
_, rvec, tvec = cv2.solvePnP(objp, corners2, mtx, dist)

In [4]:
#Rodrigues 표현식
print(rvec)

[[ 0.00165586]
 [-0.10786186]
 [ 1.56857997]]


In [5]:
# 회전벡터를 rotation Matrix 폼으로 변경
rotM = cv2.Rodrigues(rvec)[0]

In [6]:
print(rotM)

[[-0.00148754 -0.9977148  -0.06754966]
 [ 0.99757009  0.00322459 -0.06959536]
 [ 0.06965414 -0.06748905  0.99528565]]


In [7]:
print(tvec)

[[ 1.1501963 ]
 [-2.29107472]
 [ 7.13988908]]


In [8]:
#4x4 Matrix형태로 변환
A = np.hstack([rotM, tvec])
A = np.vstack([A, [0, 0, 0, 1]])

In [9]:
print(A)

[[-1.48753705e-03 -9.97714804e-01 -6.75496644e-02  1.15019630e+00]
 [ 9.97570091e-01  3.22458982e-03 -6.95953639e-02 -2.29107472e+00]
 [ 6.96541448e-02 -6.74890506e-02  9.95285652e-01  7.13988908e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
#로봇 파라미터부분

B = np.zeros((4, 4))

X = float(input("Base to TCP Px값을 입력하시오."))
Y = float(input("Base to TCP Py값을 입력하시오."))
Z = float(input("Base ti TCP Pz값을 입력하시오."))

R = input("각도는 라디안 값입니까? Y/N")

if((R == "Y") or (R == "y")):
    yaw = float(input("Robot Parameter의 Rx(yaw)값을 입력하시오."))
    pitch = float(input("Robot Parameter의 Ry(pitch)값을 입력하시오."))
    roll = float(input("Robot Parameter의 Rz(Roll)값을 입력하시오."))
    
    yaw = math.degrees(yaw)
    pitch = math.degrees(pitch)
    roll = math.degrees(roll)

else:
    yaw = float(input("Robot Parameter의 Rx(yaw)값을 입력하시오."))
    pitch = float(input("Robot Parameter의 Ry(pitch)값을 입력하시오."))
    roll = float(input("Robot Parameter의 Rz(Roll)값을 입력하시오."))

B = Robot_Matrix.robot_Mat(X, Y, Z, roll, pitch, yaw)

B = np.matrix(B)

print(B)

Base to TCP Px값을 입력하시오.1
Base to TCP Py값을 입력하시오.2
Base ti TCP Pz값을 입력하시오.3
각도는 라디안 값입니까? Y/Ny
Robot Parameter의 Rx(yaw)값을 입력하시오.10
Robot Parameter의 Ry(pitch)값을 입력하시오.20
Robot Parameter의 Rz(Roll)값을 입력하시오.30
[[ 0.65757612 -0.43334854 -0.61628134  1.        ]
 [ 0.29539539 -1.10957974  0.74004886  2.        ]
 [-0.6930622   0.66868513 -0.26930465  3.        ]
 [ 0.          0.          0.          1.        ]]


In [ ]:
#AX = XB, 정확히는 AX + XB = C와 같은 형태의 Matrix 방정식을 실베스터 방정식이라고 한다.
#다만, 이는 현재, 존재하는 Method가 없어, 직접 제작
D = sylvester.syl(A, B)

print(D)

F = np.zeros((12, 1))

F[3] = -A[0][3]
F[7] = -A[1][3]
F[11] = -A[2][3]

print(F)

In [ ]:
X = np.linalg.solve(D, F)
print(X)

In [ ]:
Result = np.zeros((4, 4))

k = 0

for i in range(4):
    for j in range(4):
        if(k <=11):
            Result[i][j] = X[k]
        k = k + 1
        
Result[3][3] = 1
print(Result)